# Credits

This is heavily influenced or copied from https://github.com/pytorch/tutorials

# Autograd: automatic differentiation

Central to all neural networks in PyTorch is the ``autograd`` package.
Let’s first briefly visit this, and we will then go to training our first neural network.

The `autograd` package provides automatic differentiation for all operations on Tensors.
It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

Let us see this in more simple terms with some examples.

## 1. Tensor

`torch.Tensor` is the central class of the package. Setting the attribute `.requires_grad` to `True` will make the tensor "record" all operations on it. When you finish your computation you can call `.backward()` and have all the gradients computed automatically. The gradient for this tensor will be accumulated into the `.grad` attribute.

![autograd.Variable](../static_files/autograd-variable.png)

There’s one more class which is very important for autograd implementation - a `Function`.

`Tensor` and `Function` are interconnected and build up an acyclic graph, that encodes a complete history of computation. Each tensor has a `.grad_fn` attribute that references a `Function` that has created the `Tensor` (except for Tensors created by the user - their `grad_fn` is `None`).

If you want to compute the derivatives, you can call `.backward()` on a Tensor. If `Tensor` is a scalar (i.e. it holds a one element data), you don’t need to specify any arguments to backward(), however if it has more elements, you need to specify a `gradient` argument that is a tensor of matching shape.

In [75]:
import torch

Create a tensor

In [76]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Do a tensor operation:

In [27]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


`y` was created as a result of an operation, so it has a `grad_fn`.

In [28]:
print(y.grad_fn)

Do more operations on y

In [29]:
z = y * y * 3
out = z.mean()

print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


# Assignments

1. Create a Tensor that `requires_grad` of size (5, 5)
2. Sum the values in the Tensor

In [30]:
t = torch.Tensor(5,5).requires_grad_()
t

tensor([[1.5766e-19, 1.0256e-08, 1.6802e-04, 3.0613e+12, 4.3354e-08],
        [1.0374e-08, 6.6287e-10, 1.2892e+16, 2.1707e-18, 7.0952e+22],
        [1.7748e+28, 1.8176e+31, 7.2708e+31, 5.0778e+31, 3.2608e-12],
        [1.7728e+28, 7.0367e+22, 2.1715e-18, 3.3064e-09, 1.0625e+21],
        [2.5643e-09, 2.6219e-09, 1.6536e-04, 1.0243e-11, 8.3654e+20]],
       requires_grad=True)

In [31]:
torch.sum(t)


tensor(1.4170e+32, grad_fn=<SumBackward0>)

## 2. Gradients

Let’s backprop now. Because `out` contains a single scalar, `out.backward()` is equivalent to `out.backward(torch.tensor([1.0]))`

In [32]:
out.backward()

Print gradients d(out)/dx

In [33]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


You should have a matrix of `4.5`. Let’s denote the tensor `out` with $o$.

We have:
$o = \frac{1}{4}\sum_i z_i$, $z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.

Therefore, $\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$,
hence $\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.

You can do many crazy things with autograd!

In [78]:
x

tensor([-1.1553,  1.3553,  1.6248], requires_grad=True)

In [85]:
x = torch.randn(3, requires_grad=True)

y = x * 2
#hile y.data.norm() < 1000:
#    y = y * 2

print(y)

tensor([ 2.4857, -3.5749,  0.3565], grad_fn=<MulBackward0>)


In [86]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

tensor([2.0000e-01, 2.0000e+00, 2.0000e-04])


**Read later** \
*Documentation* \
`Tensor`: https://pytorch.org/docs/stable/tensors.html \
`Function`: http://pytorch.org/docs/autograd

# Assignments

1. Define a tensor and set `requires_grad` to `True`
3. Multiply the tensor by 2 and assign the result to a new python variable (i.e. `x = result`)
4. Sum the variable's elements and assign to a new python variable
5. Print the gradients of all the variables
6. Now perform a backward pass on the last variable (NOTE: for each new python variable that you define, call `.retain_grad()`)
7. Print all gradients again
  - what did you notice?

In [67]:
x = torch.Tensor(5,5)
x.requires_grad_()
x.retain_grad()

In [68]:
result = torch.mul(x,2)
result.retain_grad()

In [69]:
new = torch.sum(result)
new.retain_grad()
new

tensor(4.7094e-37, grad_fn=<SumBackward0>)

In [70]:
print(x.grad)
print(result.grad)
print(new.grad)


None
None
None


In [71]:
new.backward()

In [72]:
print(x.grad)
print(result.grad)
print(new.grad)


tensor([[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]])
tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])
tensor(1.)
